# 데이터 로드 

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
from glob import glob 
import os 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xvf '/content/drive/MyDrive/공유용/train_features.tar'

In [ ]:
csvs_dir = glob('/content/katl/*.bz2')
csvs_dir.sort()
csvs_dir = pd.DataFrame(csvs_dir,columns=['dir'])
csvs_name = list(csvs_dir['dir'].apply(lambda x : x.split('katl/')[-1].split('.csv')[0].split('_',maxsplit=1)[-1]))

In [32]:
def make_dir(dir,df):
  csv_dir = df[df['dir'].apply(lambda x : dir in x )].values[0][0]
  return csv_dir 

In [ ]:
for csv_name in tqdm(csvs_name):
  exec(f"{csv_name} = pd.read_csv(make_dir('{csv_name}',csvs_dir))")

# Baseline 

In [112]:
from pathlib import Path
from typing import Sequence, Tuple
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
import seaborn as sns 

## Get Data 
- 필요한 데이터는 Training features, Training labels , Submission format 

In [113]:
DATA_PATH = Path("../content/")

In [114]:
airport_directories = sorted(path for path in DATA_PATH.glob("k*"))
airport_directories

[PosixPath('../content/katl'),
 PosixPath('../content/kclt'),
 PosixPath('../content/kden'),
 PosixPath('../content/kdfw'),
 PosixPath('../content/kjfk'),
 PosixPath('../content/kmem'),
 PosixPath('../content/kmia'),
 PosixPath('../content/kord'),
 PosixPath('../content/kphx'),
 PosixPath('../content/ksea')]

10개의 공항 별로 폴더가 있음 

In [115]:
airport_directory = airport_directories[0]
data_files = list(airport_directory.glob("*"))
data_files

[PosixPath('../content/katl/katl_arrival_runway.csv.bz2'),
 PosixPath('../content/katl/katl_tfm_estimated_runway_arrival_time.csv.bz2'),
 PosixPath('../content/katl/katl_airport_config.csv.bz2'),
 PosixPath('../content/katl/katl_departure_runway.csv.bz2'),
 PosixPath('../content/katl/katl_etd.csv.bz2'),
 PosixPath('../content/katl/katl_first_position.csv.bz2'),
 PosixPath('../content/katl/katl_mfs_stand_arrival_time.csv.bz2'),
 PosixPath('../content/katl/katl_mfs_stand_departure_time.csv.bz2'),
 PosixPath('../content/katl/katl_mfs_runway_departure_time.csv.bz2'),
 PosixPath('../content/katl/katl_mfs_runway_arrival_time.csv.bz2'),
 PosixPath('../content/katl/katl_tbfm_scheduled_runway_arrival_time.csv.bz2'),
 PosixPath('../content/katl/katl_lamp.csv.bz2')]

각 공항 폴더 안에는 12개의 csv 파일 존재, 이 중 주로 사용할 것은 "katl_airport_config.csv"
- training 기간 중 실제 configuration이 담겨 있음 

In [121]:
#파일 패스 추출해서 csv로 여는 함수 
def read_airport_configs(airport_directory: Path) -> Tuple[str, pd.DataFrame]:
    """Reads the airport configuration features for a given airport data directory."""
    airport_code = airport_directory.name
    filename = f"{airport_code}_airport_config.csv.bz2"
    filepath = airport_directory / filename
    airport_config_df = pd.read_csv(filepath, parse_dates=["timestamp"])
    return airport_code, airport_config_df

In [122]:
#dict 형태로 공항 별 config 저장 함 
airport_config_df_map = {}

for airport_directory in sorted(airport_directories):
    airport_code, airport_config_df = read_airport_configs(airport_directory)
    print(airport_code)
    airport_config_df_map[airport_code] = airport_config_df

katl
kclt
kden
kdfw
kjfk
kmem
kmia
kord
kphx
ksea


## Read The Training Labels 

In [128]:
open_train_labels = pd.read_csv('/content/drive/MyDrive/공유용/open_train_labels (1).csv.bz2', parse_dates=["timestamp"])
open_train_labels.shape

(22640544, 5)

In [129]:
open_train_labels.head()

,airport,timestamp,lookahead,config,active
0,katl,2020-11-06 23:00:00,30,katl:D_10_8L_A_10_8L,0.0
1,katl,2020-11-06 23:00:00,30,katl:D_10_8R_9L_A_10_8L_9R,0.0
2,katl,2020-11-06 23:00:00,30,katl:D_10_8R_A_10_8R,0.0
3,katl,2020-11-06 23:00:00,30,katl:D_26L_27L_A_26R_27L_28,0.0
4,katl,2020-11-06 23:00:00,30,katl:D_26L_27R_28_A_26R_27L_28,0.0


**open_training_labels Columns 별 설명**
- airport : 공항 코드 
- timestamp : 예측이 생성된 시간 
- lookahead : 예측하고자 하는 시간 
- config : 가능한 configuration 
- active : 해당 시간에 실제로 활성 상태였는지 

What to do : 활성 상태의 값이 0과 1 사이의 확률 

## Get A sample Frame 
-open_train_labels에 대해 생각할 수 있는 한 가지 방법은 "예측 프레임"으로, 각 프레임은 특정 시간에 주어진 공항에 대한 예측을 나타냄 